In [ ]:
# 코드 전체를 이해하기보다는 모듈 단위로 쪼개서 메인을 먼저 찾아라
# 함수 안에 속해있지 않으면 다 메인
# 코드가 여러 줄인 것들은 함수로 빼놔라

In [1]:
from sklearn import svm, metrics   # svm은 분류모형일 때 쓸 수 있다.
import glob, os.path, re, json
# glob=모든파일로드
# os.path=문자열로 전달된것을 경로형식으로 전달
# re: regular expression, 정규표현식
# json=json형식으로 빼준다. 다음에 노트북 켜서 쓸 때도 정보가 날아가지 않고 그대로 쓸 수 있게 key, value형식으로 ...

#### 머신러닝에 사용할 데이터 정제

In [3]:
### 3. 텍스트의 음절 나눠서 문자비율 구하고 레이블값 지정
def check_freq(fname):
    with open(fname, "r", encoding='utf-8') as f:   # 파일을 읽어들이는 작업. 빨대꽂기
        text = f.read()   # 텍스트 읽어들이기 한 바이트씩... 줄바꿈이 굳이 필요 없다. 어차피 음절 단위로 쪼개니까 -> 그래서 readLines아니고 read
    text = text.lower()   # 대문자->소문자 어차피 이 프로젝트에서는 대소 구분 필요없어서
    
    # 숫자세기변수(cnt) 초기화하기
    cnt = [0 for n in range(0,26)]   # 0부터 26까지를 다 0으로 바꿔서 배열에 담아라 [0, 0,......0]
    code_a = ord("a")  # ord: 아스키문자 a를 숫자형태의 데이터로   # 'a' = 97
    code_z = ord("z")  # 'z' = 122
    
    # 알파벳 출현횟수 구하기
    for ch in text:
        n = ord(ch)   # n - 97
        if code_a <= n <= code_z:    # <= <= 사잇값. 유효한 값 사이에 있는지 확인. 이 범위에 안들면 마이너스가 되거나 초과. 여기서 구둣점이나 역슬래시 따옴표 숫자 등이 제외된다.
            cnt[n - code_a] += 1
        
    # 정규화: 각 문장길이당 문자비율이 공평해야 하므로. 
    total = sum(cnt)
    freq = list(map(lambda n : n / total, cnt))  # 행이 여러개인 벡터면 굳이 안 써도 되나, 이것은 칼럼이 여러개라 for문 돌려야한다.
    # map은 list의 요소를 지정된 함수로 처리
    #print(freq)
            
    # 레이블값(정답) 구하기 -> 파일명이 정답이다
    name = os.path.basename(fname)   # basename은 앞의 경로 싹 빠지고 확장자명포함 파일이름만 나온다.
    #print(name)
    lang = re.match(r'^[a-z]{2,}', name).group()  # 한글이 써있어도 유니코드 형식으로 맵핑해달라 # a-z까지 두글자로 시작(^). match는 맞는지 안맞는지 보기. group은 잘맞는것 가져오기
    return freq, lang    


### 2. 해당 경로의 모든 파일을 하나하나 함수처리하고(for문) 필요한 값 리턴 
def load_files(path):
    freqs = []   # count 저장해놓기 위해
    labels = []
    file_list = glob.glob(path)  #path에 있는 파일을 다 보여달라
    for fname in file_list:    
        r = check_freq(fname)     # 매 파일마다 불러와서 음절단위로 쪼개야 하니 함수로 만든다.
        freqs.append(r[0])  # [0.223,3343] 횟수를 리스트로 갖고있음. 여러개의 파일정보 2차원으로 다 가짐. 카운트횟수 저장해놓기
        labels.append(r[1])  # 'en' 1차원의 labels 문자열 리스트. 저장을 위해.
    return {"freqs": freqs, "labels":labels}


### 1. main - 해당 경로의 train, text 데이터 모두 불러와서 json형식으로 저장해라
# main을 제일 먼저 만든 후 필요한 함수는 윗줄에 추가한다.
train = load_files('./lang/train/*.txt')   # txt만 읽을 수 있게 조정
test = load_files('./lang/test/*.txt')
# test['freqs']  # test는 딕셔너리 test['freqs']는 2차원 배열
# json형식으로 만들기(매번 쓸 때마다 코드를 다시 실행할 필요 X)
with open("./lang/freq.json", "w", encoding="utf-8") as fp:
    json.dump([train, test], fp)   #dump는 write나 save같은 말 . json형식으로 저장해라

#### 머신러닝 기법 활용(SVM)

In [15]:
# 학습하기
clf = svm.SVC()    # SVC: classification 분류하는 svm 모델
clf.fit(train["freqs"], train["labels"])  # regression은 SVR
# fit은 feature하고 label 반드시 구분해서 전달

# kernel='rbt' -> 가우시안


C:\Users\student\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [25]:
# gamma값 바꿔서 다시 해보기

In [16]:
# 예측하기
predict = clf.predict(test["freqs"])
predict

array(['en', 'en', 'fr', 'fr', 'en', 'id', 'tl', 'tl'], dtype='<U2')

In [17]:
# 결과테스트하기
ac_score = metrics.accuracy_score(test["labels"],predict)
cl_report = metrics.classification_report(test["labels"],predict)
print("정답률 =", ac_score)
print("리포트 =")
print(cl_report)

# precision(예측치기준), recall(실측치기준), f1-score 기억해두기
# 인도네시아어는 반밖에 못맞춘것....-> 데이터를 준비할때 인도네시아스러운 자료로 바꿔볼까?
# 인도네시아로 예측하는 것은 100% 정확, but 실제데이터는 반밖에 못맞춤춤
# f1 score는 조화평균. precision과 recall을 둘다 본다.

# 제일 중요한것은 결국 accuracy score!

정답률 = 0.875
리포트 =
              precision    recall  f1-score   support

          en       0.67      1.00      0.80         2
          fr       1.00      1.00      1.00         2
          id       1.00      0.50      0.67         2
          tl       1.00      1.00      1.00         2

    accuracy                           0.88         8
   macro avg       0.92      0.88      0.87         8
weighted avg       0.92      0.88      0.87         8



In [ ]:
# lang-plot.py 로 그래프 출력해보기


In [ ]:
# lang-Webapp.py 로 웹 만들기


In [24]:
# lang-train-save.py
# 각 언어의 출현 빈도데이터(JSON) 읽어들이기
import json #json은 json형식, joblib은 pickle에 특화
from sklearn.externals import joblib
with open("./lang/freq.json","r",encoding="utf-8") as fp:
    d = json.load(fp)
    data = d[0]

# 학습데이터 저장하기
joblib.dump(clf, "./lang/freq.pkl")
print("ok")

ok


C:\Users\student\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [ ]:
# 서버구동
anaconda -> python -m http.server --cgi 8080

화면에 띄우기
http://localhost:8080/cgi-bin/lang-Webapp.py
외국어 쓴 다음 판정 누르면 판별된다.